# Galactic models comparison

In [ ]:
from healpy.newvisufunc import projview
from healpy.rotator import Rotator
import healpy as hp
import traceback

import pandas as pd
import numpy as np
from scipy.integrate import quad
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import itertools

import plotly.graph_objects as go
import plotly.express as px

import os
import sys
from radiocalibrationtoolkit import *

In [ ]:
# This ensures Plotly output works in multiple places:
# plotly_mimetype: VS Code notebook UI
# notebook: "Jupyter: Export to HTML" command in VS Code
# See https://plotly.com/python/renderers/#multiple-renderers
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"

In [ ]:
# some global plot settings
plt.rcParams["axes.titleweight"] = "bold"
plt.rcParams["axes.labelweight"] = "bold"
plt.rcParams["font.weight"] = "bold"
plt.rcParams['font.size'] = 16
plt.rcParams['legend.fontsize']= 14

plt.rcParams['xtick.major.width']= 2
plt.rcParams['ytick.major.width']= 2

plt.rcParams['xtick.major.size']= 5
plt.rcParams['ytick.major.size']= 5

plt.rcParams['xtick.labelsize']= 14
plt.rcParams['ytick.labelsize']= 14

In [ ]:
# Prepare objects
lfmap = LFmap()
lfss = LowFrequencySkyModel(freq_unit="MHz")
gsm2008 = GlobalSkyModel(freq_unit="MHz")
gsm2016 = GlobalSkyModel2016(freq_unit="MHz")
haslam = HaslamSkyModel(freq_unit="MHz", spectral_index=-2.53)
ssm = SSM()
gmoss = GMOSS()
ulsa_fdi = ULSA(index_type='freq_dependent_index')
ulsa_ci = ULSA(index_type='constant_index')
ulsa_dpi = ULSA(index_type='direction_dependent_index')

In [ ]:
frequency_MHz = 45

lfmap_map = lfmap.generate(frequency_MHz)
lfss_map = lfss.generate(frequency_MHz)
gsm2008_map = gsm2008.generate(frequency_MHz)
gsm2016_map = gsm2016.generate(frequency_MHz)
haslam_map = haslam.generate(frequency_MHz)
ssm_map = ssm.generate(frequency_MHz)
gmoss_map = gmoss.generate(frequency_MHz)
ulsa_fdi_map = ulsa_fdi.generate(frequency_MHz)
ulsa_ci_map = ulsa_ci.generate(frequency_MHz)
ulsa_dpi_map = ulsa_dpi.generate(frequency_MHz)

In [ ]:
# Check size of the arrays and NSIDE number and angular resolution

map_list = [lfmap_map, lfss_map, gsm2008_map, gsm2016_map, haslam_map, ssm_map, gmoss_map, ulsa_fdi_map, ulsa_ci_map, ulsa_dpi_map]
[print_map_properties(m) for m in map_list]

In [ ]:
# convert to same NSIDE, NSIDE is always some power of two
new_nside = 64
lfmap_map_N = hp.ma(hp.pixelfunc.ud_grade(lfmap_map, new_nside))
lfss_map_N = hp.ma(hp.pixelfunc.ud_grade(lfss_map, new_nside))
gsm2008_map_N = hp.ma(hp.pixelfunc.ud_grade(gsm2008_map, new_nside))
gsm2016_map_N = hp.ma(hp.pixelfunc.ud_grade(gsm2016_map, new_nside))
ssm_map_N = hp.ma(hp.pixelfunc.ud_grade(ssm_map, new_nside))
haslam_map_N = hp.ma(hp.pixelfunc.ud_grade(haslam_map, new_nside))
gmoss_map_N = hp.ma(hp.pixelfunc.ud_grade(gmoss_map, new_nside))
ulsa_fdi_map_N = hp.ma(hp.pixelfunc.ud_grade(ulsa_fdi_map, new_nside))
ulsa_ci_map_N = hp.ma(hp.pixelfunc.ud_grade(ulsa_ci_map, new_nside))
ulsa_dpi_map_N = hp.ma(hp.pixelfunc.ud_grade(ulsa_dpi_map, new_nside))

In [ ]:
map_dict = {
    "LFSS": lfss_map_N,
    "GSM08": gsm2008_map_N,
    "GSM16": gsm2016_map_N,
    "Haslam": haslam_map_N,
    "LFmap": lfmap_map_N,
    "SSM": ssm_map_N,
    "GMOSS": gmoss_map_N,
    "ULSA": ulsa_fdi_map_N,
    # "ULSA2": ulsa_ci_map_N,
    # "ULSA3": ulsa_dpi_map_N,
}

ulsa_map_dict = {
    "ULSA FDI": ulsa_fdi_map_N,
    "ULSA CI": ulsa_ci_map_N,
    "ULSA DPI": ulsa_dpi_map_N,
}


In [ ]:
# Print the maps in the dictionary
[(key, map_dict[key]) for key in map_dict]

## Galactic coordinates

In [ ]:
# Prints all maps
cmap = "jet"
projection_type = "mollweide"

for key in map_dict:

    projview(
        map_dict[key],
        norm="log",
        coord=["G"],
        graticule=True,
        graticule_labels=True,
        unit="Temperature ln[K]",
        xlabel="RA",
        ylabel="DEC",
        cb_orientation="vertical",
        min=3500,
        max=35000,
        latitude_grid_spacing=30,
        projection_type=projection_type,
        title=key,
        xtick_label_color="white",
        cmap=cmap,
    )

In [ ]:
compare_maps(
    map_dict,
    main_title="Frequency: {} MHz; Compared on {:.2f}{} angular resolution".format(
        frequency_MHz, np.rad2deg(hp.pixelfunc.nside2resol(new_nside)), chr(176)
    ),
)

In [ ]:
compare_maps(
    ulsa_map_dict,
    main_title="Frequency: {} MHz; Compared on {:.2f}{} angular resolution".format(
        frequency_MHz, np.rad2deg(hp.pixelfunc.nside2resol(new_nside)), chr(176)
    ),
    figsize=(6, 4.5)
)

### Compare at all frequencies

In [ ]:
f_range = range(30, 85, 5)
for frequency_MHz in f_range:
    # LFmap is by default in Celestial coordinates, to make things simpler, we rotate it to Galactic coordinates at the creation stage.
    lfmap_map = hp.rotator.Rotator.rotate_map_pixel(
        Rotator(coord=["C", "G"]), lfmap.generate(frequency_MHz)
    )
    lfmap_map = lfmap.generate(frequency_MHz)
    lfss_map = lfss.generate(frequency_MHz)
    gsm2008_map = gsm2008.generate(frequency_MHz)
    gsm2016_map = gsm2016.generate(frequency_MHz)
    haslam_map = haslam.generate(frequency_MHz)
    ssm_map = ssm.generate(frequency_MHz)
    gmoss_map = gmoss.generate(frequency_MHz)
    ulsa_fdi_map = ulsa_fdi.generate(frequency_MHz)
    ulsa_ci_map = ulsa_ci.generate(frequency_MHz)
    ulsa_dpi_map = ulsa_dpi.generate(frequency_MHz)

    # convert to same NSIDE
    new_nside = 64
    lfmap_map_N = hp.ma(hp.pixelfunc.ud_grade(lfmap_map, new_nside))
    lfss_map_N = hp.ma(hp.pixelfunc.ud_grade(lfss_map, new_nside))
    gsm2008_map_N = hp.ma(hp.pixelfunc.ud_grade(gsm2008_map, new_nside))
    gsm2016_map_N = hp.ma(hp.pixelfunc.ud_grade(gsm2016_map, new_nside))
    ssm_map_N = hp.ma(hp.pixelfunc.ud_grade(ssm_map, new_nside))
    haslam_map_N = hp.ma(hp.pixelfunc.ud_grade(haslam_map, new_nside))
    gmoss_map_N = hp.ma(hp.pixelfunc.ud_grade(gmoss_map, new_nside))
    ulsa_fdi_map_N = hp.ma(hp.pixelfunc.ud_grade(ulsa_fdi_map, new_nside))
    ulsa_ci_map_N = hp.ma(hp.pixelfunc.ud_grade(ulsa_ci_map, new_nside))
    ulsa_dpi_map_N = hp.ma(hp.pixelfunc.ud_grade(ulsa_dpi_map, new_nside))

    map_temp_dict = {
        "LFSS": lfss_map_N,
        "GSM08": gsm2008_map_N,
        "GSM16": gsm2016_map_N,
        "Haslam": haslam_map_N,
        "LFmap": lfmap_map_N,
        "SSM": ssm_map_N,
        "GMOSS": gmoss_map_N,
        "ULSA": ulsa_fdi_map_N,
        # "ULSA2": ulsa_ci_map_N,
        # "ULSA3": ulsa_dpi_map_N,
    }


    compare_dict = compare_maps(
        map_temp_dict,
        show_plot_comparison=False,
        verbose=False
    )

    if frequency_MHz == list(f_range)[0]:
        compare_DF = pd.DataFrame(compare_dict).T
        compare_DF.columns = [list(f_range)[0]]
    else:
        compare_DF[frequency_MHz] = pd.DataFrame(compare_dict).T.values

In [ ]:
fig = px.imshow(
    compare_DF,
    width=500,
    height=600,
    color_continuous_scale="jet",
    zmin=-0.3,
    zmax=0.3,
    labels={"x": "<b>frequency [MHz]</b>"},
)
fig.update_layout(
    margin=dict(l=20, r=120, t=20, b=20),
    xaxis={"nticks": 5},
    font=dict(family="Arial Black", size=15, color="black"),
)
fig.update_yaxes(tickprefix="<b>", ticksuffix="</b>")
fig.update_xaxes(
    tickprefix="<b>", ticksuffix="</b>", tick0=30, dtick=10, tickfont=dict(size=18)
)

fig.update_coloraxes(
    colorbar=dict(
        tickprefix="<b>",
        ticksuffix="</b>",
        tickfont=dict(size=18),
    )
)

fig.add_annotation(
    go.layout.Annotation(
        text='<b>RDIS</b>',
        x=1.4,
        y=0.5,
        xref='paper',
        yref='paper',
        showarrow=False,
        font=dict(size=18),
        textangle=-90,
        align='center',
        valign='middle',
    )
)

fig.show()

In [ ]:
fig, ax = plt.subplots()
ax.set_title('all \n')

# temp = compare_DF.std(axis=0)
labels = compare_DF.columns.astype(float)
ax.errorbar(
    labels,
    compare_DF.mean(axis=0).values,
    yerr=compare_DF.std(axis=0),
    marker=".",
    linestyle="",
    markersize=10,
)

ax.axes.axhline(0, ls="--", c="r")
ax.set_xlabel("frequency [MHz]")
ax.set_ylabel("<RDIS>$_{\mathrm{models}}$")
ax.set_ylim(-0.15, 0.15)

fig, ax = plt.subplots()
ax.set_title('without LFSS\n')

ax.errorbar(
    labels,
    compare_DF.iloc[7:,:].mean(axis=0).values,
    yerr=compare_DF.std(axis=0),
    marker=".",
    linestyle="",
    markersize=10,
)

ax.axes.axhline(0, ls="--", c="r")
ax.set_xlabel("frequency [MHz]")
ax.set_ylabel("<RDIS>$_{\mathrm{models}}$")
ax.set_ylim(-0.15, 0.15)


fig, ax = plt.subplots()
ax.set_title('without LFSS and GSM08\n')

ax.errorbar(
    labels,
    compare_DF.iloc[13:,:].mean(axis=0).values,
    yerr=compare_DF.std(axis=0),
    marker=".",
    linestyle="",
    markersize=10,
)

ax.axes.axhline(0, ls="--", c="r")
ax.set_xlabel("frequency [MHz]")
ax.set_ylabel("<RDIS>$_{\mathrm{models}}$")
ax.set_ylim(-0.15, 0.15)

In [ ]:
compare_DF.iloc[12:,:]

In [ ]:
df = compare_DF.copy(deep=True)
df = compare_DF.iloc[::-1,:]
fig, ax = plt.subplots(figsize=(6, 8))
labels = df.index.values
err = df.std(axis=1)

pos = range(labels.size)
ax.errorbar(
    df.mean(axis=1), pos, xerr=err, marker=".", linestyle="", markersize=10
)
ax.set_yticks(pos, labels=labels)
ax.set_xlabel("<RDIS>$_{\\nu}$")

In [ ]:
input_data1 = compare_DF.values.flatten()
input_data2 = compare_DF.iloc[7:, :].values.flatten()

bins = np.linspace(-0.5, 0.5, 20)
fig, ax = plt.subplots()
fig.suptitle('')
ax.hist(input_data1, bins=bins, density=True, alpha=0.5)
ax.hist(input_data2, bins=bins, density=True, alpha=0.5)

ax.set_xlabel("RDIS")
ax.set_ylabel("entries")

fig, ax = plt.subplots()
# fig.suptitle('Blue: all models, Red: without LFSS')
ax.set_xlabel("RDIS")
apply_KDE(input_data1, bounds=[bins[0], bins[-1]], data_in_relative_values=True)
apply_KDE(input_data2, bounds=[bins[0], bins[-1]], data_in_relative_values=True)

In [ ]:
display(input_data1.size)
display(input_data2.size)

## Local obsever

In [ ]:
lst = 18
latitude = -35.206667
frequency_MHz = 45
main_title = "Frequency: {} MHz; Compared on {:.2f}{} angular resolution; LST:{}; Latitude:{:.2f}".format(
    frequency_MHz, np.rad2deg(hp.pixelfunc.nside2resol(new_nside)), chr(176), lst, latitude
)

# Local coordinates at LST time "LSTtime" at latitude "latitude"
rotation_parameters = [(180 + (lst * 15)) % 360, -(latitude - 90)]

In [ ]:
# some plot settings
cmap = "jet"
projection_type = "mollweide"

In [ ]:
# Prints all maps
for key in map_dict:

    projview(
        map_dict[key],
        norm="log",
        coord=['G','C'],
        rot=rotation_parameters,
        graticule=True,
        graticule_labels=True,
        unit="Temperature ln[K]",
        xlabel="azimuth",
        ylabel="altitude",
        cb_orientation="vertical",
        min=3500,
        max=35000,
        latitude_grid_spacing=30,
        projection_type=projection_type,
        title=key,
        xtick_label_color="white",
        cmap=cmap,
    )

In [ ]:
# Create mask, at local coordinates we see only a hemisphere
mask = create_local_mask(new_nside, rotation_parameters)

In [ ]:
# Test
m = map_dict['LFmap'].copy()
m.mask = mask

projview(
    m,
    norm="log",
    coord=["G", 'C'],
    rot=rotation_parameters,
    graticule=True,
    graticule_labels=True,
    unit="Temperature ln[K]",
    xlabel="azimuth",
    ylabel="altitude",
    cb_orientation="vertical",
    min=3500,
    max=35000,
    latitude_grid_spacing=30,
    xtick_label_color="white",
    title=main_title,
    cmap=cmap,
)

projview(
    m,
    norm="log",
    # coord=["G"],
    # rot=False,
    graticule=True,
    graticule_labels=True,
    unit="Temperature ln[K]",
    xlabel="RA",
    ylabel="DEC",
    cb_orientation="vertical",
    min=3500,
    max=35000,
    latitude_grid_spacing=30,
    xtick_label_color="white",
    title=main_title,
    cmap=cmap,
)

### In local coordinates

In [ ]:
compare_maps(
    map_dict, rotation_parameters=rotation_parameters, coord=['G','C'], main_title=main_title, mask=mask
)

### In galactic coordinates (masked)

In [ ]:
compare_maps(map_dict, rotation_parameters=False, main_title=main_title, mask=mask)

### Complex comparison

In [ ]:
f_range = range(30, 85, 5)
lst_range = range(0, 24, 2)
# f_range = range(30, 45, 5)
# lst_range = range(0, 6, 2)
compare_DF = pd.DataFrame(columns=['label', 'frequency [MHz]', 'LST', 'ratio'])

for frequency_MHz in f_range:
    lfmap_map = hp.rotator.Rotator.rotate_map_pixel(
        Rotator(coord=["C", "G"]), lfmap.generate(frequency_MHz)
    )
    lfmap_map = lfmap.generate(frequency_MHz)
    lfss_map = lfss.generate(frequency_MHz)
    gsm2008_map = gsm2008.generate(frequency_MHz)
    gsm2016_map = gsm2016.generate(frequency_MHz)
    haslam_map = haslam.generate(frequency_MHz)
    ssm_map = ssm.generate(frequency_MHz)
    gmoss_map = gmoss.generate(frequency_MHz)
    ulsa_fdi_map = ulsa_fdi.generate(frequency_MHz)
    ulsa_ci_map = ulsa_ci.generate(frequency_MHz)
    ulsa_dpi_map = ulsa_dpi.generate(frequency_MHz)

    # convert to same NSIDE
    new_nside = 64
    lfmap_map_N = hp.ma(hp.pixelfunc.ud_grade(lfmap_map, new_nside))
    lfss_map_N = hp.ma(hp.pixelfunc.ud_grade(lfss_map, new_nside))
    gsm2008_map_N = hp.ma(hp.pixelfunc.ud_grade(gsm2008_map, new_nside))
    gsm2016_map_N = hp.ma(hp.pixelfunc.ud_grade(gsm2016_map, new_nside))
    ssm_map_N = hp.ma(hp.pixelfunc.ud_grade(ssm_map, new_nside))
    haslam_map_N = hp.ma(hp.pixelfunc.ud_grade(haslam_map, new_nside))
    gmoss_map_N = hp.ma(hp.pixelfunc.ud_grade(gmoss_map, new_nside))
    ulsa_fdi_map_N = hp.ma(hp.pixelfunc.ud_grade(ulsa_fdi_map, new_nside))
    ulsa_ci_map_N = hp.ma(hp.pixelfunc.ud_grade(ulsa_ci_map, new_nside))
    ulsa_dpi_map_N = hp.ma(hp.pixelfunc.ud_grade(ulsa_dpi_map, new_nside))

    map_temp_dict = {
        "LFSS": lfss_map_N,
        "GSM08": gsm2008_map_N,
        "GSM16": gsm2016_map_N,
        "Haslam": haslam_map_N,
        "LFmap": lfmap_map_N,
        "SSM": ssm_map_N,
        "GMOSS": gmoss_map_N,
        "ULSA": ulsa_fdi_map_N,
        # "ULSA2": ulsa_ci_map_N,
        # "ULSA3": ulsa_dpi_map_N,
    }
    
    for lst in lst_range:
        rotation_parameters = [(180 + (lst * 15)) % 360, -(latitude - 90)]
        mask = create_local_mask(new_nside, rotation_parameters)

        compare_dict = compare_maps(
            map_temp_dict,
            rotation_parameters=False,
            mask=mask,
            show_plot_comparison=False,
            verbose=False
        )

        for key in compare_dict:
            compare_DF.loc[compare_DF.index.size] = [key, frequency_MHz, lst, compare_dict[key][0]]



In [ ]:
compare_DF
df = compare_DF.sort_values(by=['label', 'frequency [MHz]', 'LST'])
models_ratio_list = list(dict.fromkeys(df.label.values))

In [ ]:
default_color_cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
# delete red
r = default_color_cycle[3]
del default_color_cycle[3]
b='k'
# b = default_color_cycle[0]
# del default_color_cycle[0]

fig, ax = plt.subplots(1,2, figsize=(10,4))

for i, frequency in enumerate(f_range):
    for models_ratio in models_ratio_list:
        temp = df[(df['label']==models_ratio) & (df['frequency [MHz]']==frequency)]
        if 'LFSS' in models_ratio:
            ax[0].plot(temp.LST.values, temp.ratio.values, c=r, alpha=1)#, zorder=len(f_range)+i)
        elif 'GSM08' in models_ratio:
            ax[0].plot(temp.LST.values, temp.ratio.values, alpha=0.5)#, zorder=len(f_range)+i)
        else:
            ax[0].plot(temp.LST.values, temp.ratio.values, alpha=0.5)

ax[0].set_xlabel('LST')
ax[0].set_ylabel('RDIS')

for lst in lst_range:
    for models_ratio in models_ratio_list:
        temp = df[(df['label']==models_ratio) & (df['LST']==lst)]
        if 'LFSS' in models_ratio:
           ax[1].plot(temp['frequency [MHz]'].values, temp.ratio.values, c=r, alpha=1)#, zorder=len(f_range)+i)
        elif 'GSM08' in models_ratio:
           ax[1].plot(temp['frequency [MHz]'].values, temp.ratio.values, alpha=0.5)#, zorder=len(f_range)+i)
        else:
           ax[1].plot(temp['frequency [MHz]'].values, temp.ratio.values, alpha=0.5)

ax[1].set_xlabel('frequency [MHz]')
ax[1].set_ylabel('RDIS')
fig.subplots_adjust(wspace=0.3)

ax[0].xaxis.set_major_locator(MultipleLocator(4))
ax[1].xaxis.set_major_locator(MultipleLocator(10))

# Reset to default color cycle
plt.rcParams['axes.prop_cycle'] = plt.rcParamsDefault['axes.prop_cycle']

In [ ]:
center_DF = pd.DataFrame()
center_DF.index = lst_range
left_DF = center_DF.copy(deep=True)
right_DF = center_DF.copy(deep=True)

bins=np.linspace(-0.5, 0.5, 20)

for frequency in f_range:
    center = []
    left = []
    right = []
    for lst in lst_range:
        data = df[(df['frequency [MHz]'] == frequency) & (df['LST'] == lst)]['ratio'].values

        xax_kde = np.linspace(bins[0], bins[-1], 2000)[:, np.newaxis]
        kde = KernelDensity(kernel="gaussian", bandwidth=0.05).fit(data[:, np.newaxis])
        log_dens = kde.score_samples(xax_kde)

        xax_ = xax_kde.flatten()

        center_i = find_index_on_CDF(log_dens, xax_kde, 0.5)
        left_i = find_index_on_CDF(log_dens, xax_kde, 0.5-0.341)
        right_i = find_index_on_CDF(log_dens, xax_kde, 0.5+0.341)

        center.append(xax_[center_i])
        left.append(xax_[left_i+1])
        right.append(xax_[right_i])
    center_DF[frequency] = center
    left_DF[frequency] = left
    right_DF[frequency] = right


In [ ]:
font = "Arial Black"

scene = dict(
    xaxis=dict(
        title="<b>frequency [MHz]</b>",
        color="black",
        tickvals=list(range(len(f_range))),
        ticktext=center_DF.columns.values,
    ),
    yaxis=dict(
        title="<b>LST [hour]</b>",
        color="black",
        tickvals=list(range(len(lst_range))),
        ticktext=center_DF.index.values.tolist(),
    ),
    zaxis=dict(title="<b><RDIS></b>", range=(-0.1, 0.05), dtick=0.05),
    aspectratio=dict(x=0.75, y=0.75, z=0.75),  # Adjust the aspect ratio as needed
    camera=dict(eye=dict(x=1.2, y=1.2, z=0.6)),  # Adjust the camera position as needed
)

# layout = go.Layout(scene=scene)

fig = go.Figure(
    data=[
        # go.Surface(z=right_DF),
        go.Surface(z=center_DF, showscale=True, opacity=0.9)
    ],
    # go.Surface(z=left_DF, showscale=False, opacity=0.9)],
)

fig.update_layout(
    scene=scene,
    margin=dict(r=50, b=10, l=10, t=10),
    height=600,
    autosize=False,
    font=dict(family=font, size=18, color="black"),
)

fig.update_traces(
    colorbar=dict(
        tickprefix="<b>",
        ticksuffix="</b>",
        title="<b><RDIS></b>",
        titleside="right",
        tickfont=dict(size=18),
        len=0.75,  # Adjust the length of the colorbar
    )
)

fig.show()
################################################################
################################################################
################################################################

fig = go.Figure(
    data=[
        go.Surface(z=(center_DF - right_DF).abs(), opacity=0.5),
        # go.Surface(z=center_DF, showscale=False, opacity=0.9)],
        go.Surface(z=(center_DF - left_DF).abs(), showscale=False, opacity=0.5),
    ],
)

scene["zaxis"] = {"title": "<b><RDIS> errors</b>", "dtick": 0.05}
fig.update_layout(
    scene=scene,
    margin=dict(r=20, b=10, l=10, t=10),
    height=600,
    autosize=False,
    font=dict(family=font, size=18, color="black"),
)

fig.update_traces(
    colorbar=dict(
        tickprefix="<b>",
        ticksuffix="</b>",
        title="<b><RDIS> errors</b>",
        titleside="right",
        tickfont=dict(size=18),
        len=0.75,  # Adjust the length of the colorbar
    )
)

fig.show()

In [ ]:
input_data = df.ratio.values
# input_data = df[(df['frequency [MHz]'] >45) & (df['frequency [MHz]'] <999)  & ((df['LST'] >=0) | (df['LST'] <=777))].ratio.values
input_data = df[(df['frequency [MHz]'] >0) & (df['frequency [MHz]'] <99)].ratio.values

########################################
fig, ax = plt.subplots()

bins=np.linspace(-0.5, 0.5, 20)
hist_data = ax.hist(input_data, bins=bins)

xax_kde = np.linspace(bins[0], bins[-1], 2000)[:, np.newaxis]
kde = KernelDensity(kernel="gaussian", bandwidth=0.05).fit(input_data[:, np.newaxis])
log_dens = kde.score_samples(xax_kde)

ax.set_xlabel('RDIS')
ax.set_ylabel('entries')
########################################

fig, ax = plt.subplots()

xax_ = xax_kde.flatten()
ax.plot(xax_, np.exp(log_dens))

center_i = find_index_on_CDF(log_dens, xax_kde, 0.5)
left_i = find_index_on_CDF(log_dens, xax_kde, 0.5-0.341)
right_i = find_index_on_CDF(log_dens, xax_kde, 0.5+0.341)

print('Test1: p0.5={}'.format(np.sum(np.exp(log_dens[:center_i+1])*np.diff(xax_)[0]) - 0.5))
print('Test2: p0.5={}'.format(np.sum(np.exp(log_dens[center_i+1:])*np.diff(xax_)[0]) - 0.5))
print('Test3: p0.5+/-0.341={}'.format(np.sum(np.exp(log_dens[left_i+1:right_i+1])*np.diff(xax_)[0]) - 2*0.341))
print(xax_[center_i], xax_[left_i+1], xax_[right_i])

ax.fill_between(
        x= xax_, 
        y1=  np.exp(log_dens), 
        where= (xax_ >= xax_[left_i+1]) & (xax_ < xax_[right_i+1]),
        color= "b",
        alpha= 0.2)

ax.axes.axvline(xax_[center_i])      
ax.set_xlabel('RDIS')
ax.set_ylabel('PDF')

textstr = '\n'.join((
    r'$\mu$={:.2f}'.format(xax_[center_i]),
    r'$\sigma_-$={:.2f}'.format(xax_[left_i+1]),
    r'$\sigma_+$={:.2f}'.format(xax_[right_i+1])))

props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

# place a text box in upper left in axes coords
ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=14,
        verticalalignment='top', bbox=props)

############################
fig, ax = plt.subplots()

ax.plot(xax_, np.cumsum(np.exp(log_dens)*np.diff(xax_)[0]))

ax.set_xlabel('RDIS')
ax.set_ylabel('CDF')

In [ ]:
input_data1 = df.ratio.values
input_data2 = df[[False if 'LFSS' in s else True for s in df.label.values]].ratio.values

fig, ax = plt.subplots()
fig.suptitle('')
ax.hist(input_data1, bins=bins, density=True, alpha=0.5)
ax.hist(input_data2, bins=bins, density=True, alpha=0.5)

ax.set_xlabel("RDIS")
ax.set_ylabel("entries")

fig, ax = plt.subplots()
# fig.suptitle('Blue: all models, Red: without LFSS')
ax.set_xlabel('RDIS')
apply_KDE(input_data1, bounds=[bins[0], bins[-1]], data_in_relative_values=True)
apply_KDE(input_data2, bounds=[bins[0], bins[-1]], data_in_relative_values=True)